In [1]:
! pip install --quiet langchain langchain_community tiktoken langchain-openai unstructured chromadb  gpt4all

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 

In [2]:
!pip install sentence-transformers -q

In [3]:
%%capture
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.20#' | sh
!sudo apt install -y neofetch

In [4]:
!neofetch

            .-/+oossssoo+/-.
        `:+ssssssssssssssssss+:`
      -+ssssssssssssssssssyyssss+-
    .ossssssssssssssssssdMMMNysssso.
   /ssssssssssshdmmNNmmyNMMMMhssssss/
  +ssssssssshmydMMMMMMMNddddyssssssss+
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/
  +sssssssssdmydMMMMMMMMddddyssssssss+
   /ssssssssssshdmNNNNmyNMMMMhssssss/
    .ossssssssssssssssssdMMMNysssso.
      -+sssssssssssssssssyyyssss+-
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.
root@c8123dac010f 
----------------- 
OS: Ubuntu 20.04.6 LTS x86_64 
Host: Google Compute Engine 
Kernel: 5.15.133+ 
Uptime: 7 mins 
Packages: 725 (dpkg) 
Shell: bash 5.0.17 
CPU: Intel Xeon (4) @ 2.000GHz 
GPU: NVIDIA Tesla T4 
GPU: NVIDIA Tesla T4 
M

In [5]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)

time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 278


In [6]:
%%capture
!ollama pull mistral:instruct

In [7]:
import pandas as pd
import torch

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from langchain.prompts import PromptTemplate
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredExcelLoader

from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain

from langchain_community.chat_models import ChatOllama
from langchain.embeddings import GPT4AllEmbeddings, HuggingFaceEmbeddings

In [8]:
# !unzip /content/MW_TGBookletsExcel.zip

In [9]:
import re
import glob

books_path = "/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/*.xlsx"
booklets = glob.glob(books_path)

def remove_unwanted(text):
    pattern = re.compile(r'\\ufdd0|[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', flags=re.UNICODE)
    clean_text = pattern.sub('', text)

    return clean_text

def add_metadata(row, booklet):
    booklet = booklet.split("/")[-1][:12]
    row['text'] = f"[{booklet}, Row Number: {str(row['row'])}] {row['text']}"    
    return row


In [10]:
import os
# os.makedirs(f"/content/cleaned_booklets/", exist_ok=True)
for booklet in booklets:
    df= pd.read_excel(booklet, names=['row', 'text'])
    df['text'] = df['text'].fillna(" ")
    kwargs_to_pass = {"booklet": booklet}
    df = df.apply(add_metadata ,**kwargs_to_pass, axis=1)
    df['text'] = df['text'].apply(lambda x: remove_unwanted(str(x)))
    booklet = booklet.split("/")[-1]
    df.to_excel(f"./{booklet}", index=False)

In [11]:
cleaned_booklets = glob.glob("./*.xlsx")
loaders = [UnstructuredExcelLoader(f"{booklet}") for booklet in cleaned_booklets]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                     chunk_overlap=20)
chunked_documents = text_splitter.split_documents(docs)

# embeddings = GPT4AllEmbeddings()

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": device}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
vectordb = Chroma.from_documents(documents=chunked_documents, embedding=embeddings, persist_directory="chroma_db")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
llm = ChatOllama(model="mistral:instruct", temperature=0)
retriever = vectordb.as_retriever(k=5)

In [14]:
template = """###Retrieve and present comprehensive and informative answers to the question, using various reasoning strategies and evidence sources. Output should include:
            
            "Answer": The concise response to the question (Try to limit answer in range of 5-90 words and stick as close as possible to retrieved data).\n\n
            "Keywords": A brief, keyword-rich text from retrieved data.\n\n
            
            ### You will output in this format, without any other text:
            
                "Answers": <>,
                "Keywords": <>
            
            ### Use words carefully, try to give more information in less words.
            ###
            Here is the retrieved data: {context}\n\n
            Question: {question}
            
            ### Always return answer and keywords both with respect to question.
            """


chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [15]:
def find_matching_paragraphs(filename, text_to_check, threshold=0.9):
    # Load the DataFrame
    df = pd.read_excel(f"/kaggle/working/{filename}",names=["paragraph", "text"]) # load cleaned xlsx files 
    df.fillna('', inplace=True)
    # Concatenate all text from the 'text' column in the DataFrame
    all_text = ' '.join(df['text'].astype(str).values.tolist())

    # Combine the provided text and all text from the DataFrame
    combined_text = [text_to_check, all_text]

    # Initialize TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the text in the DataFrame
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

    # Transform the provided text
    provided_text_tfidf = tfidf_vectorizer.transform([text_to_check])

    # Calculate cosine similarity between the provided text and each paragraph in the DataFrame
    cosine_similarities = cosine_similarity(provided_text_tfidf, tfidf_matrix).flatten()

    # Find paragraphs that meet or exceed the threshold
    matching_paragraph_indices = [i for i, score in enumerate(cosine_similarities) if score >= threshold]

    if matching_paragraph_indices:
        # Get the corresponding paragraph numbers
        matching_paragraph_numbers = df.iloc[matching_paragraph_indices]['paragraph'].tolist()
        matching_paragraph_numbers = [str(int(i)) for i in matching_paragraph_numbers]
        return ', '.join(matching_paragraph_numbers)

    else:
        # If no paragraphs meet the threshold, fallback to selecting the paragraph with the highest similarity
        closest_paragraph_index = cosine_similarities.argmax()
        closest_paragraph_number = df.iloc[closest_paragraph_index]['paragraph']
        return ', '.join([str(closest_paragraph_number)])  # Return as a list

    
def get_answer_keywords(text):
    text = text.replace("\n","").replace("Answer: ","")
    split_pattern = r"(?i)Keywords:"
    lst = re.split(split_pattern, text)
    answer, keywords = lst[0], lst[1]
    
    return answer.strip(), keywords.strip()

In [16]:
%%time
from tqdm import tqdm

test = pd.read_csv("/kaggle/input/malawi-data/Test.csv")

results = []
sources = []
keywords = []
paragraph_nums = []
for i, question in enumerate(tqdm(test["Question Text"])):
    print(question)
    output = chain.invoke(question)['result']
    print(output)
    answer, keyword = get_answer_keywords(output)
        
    docs = vectordb.similarity_search(answer)
    source = docs[0].metadata['source'].split("/")[-1]
    paragraph_num = find_matching_paragraphs(source, answer)

    results.append(answer)
    sources.append(source)
    keywords.append(keyword)
    paragraph_nums.append(paragraph_num)

  0%|          | 0/499 [00:00<?, ?it/s]

What is the definition of "unusual event"


  0%|          | 1/499 [00:08<1:14:19,  8.96s/it]

 Answers: An unusual event can be a single incident or a cluster of events that are unusual for a specific community or during a certain time period. Examples include clusters of deaths from an unknown cause, sudden illness in people with travel history out of the country, and unexpected large numbers of absences from school or sales of medicines for the same kind of illness.

Keywords: unusual event, cluster of events, community, specific time period, examples, deaths, travel history, large numbers, absences, sales, medicines, information collection, sectors.
What is Community Based Surveillance (CBS)?


  0%|          | 2/499 [00:12<48:49,  5.89s/it]  

...

 Answers: Monitoring the completeness of case reporting in IDSR involves several steps. At the district level, timely and complete monthly reporting should be monitored (8.3.1.2). Identifying problems and taking action is also important (8.3.1.3). Reporting completeness to other levels is required (8.3.1.4). Monitoring other attributes for assessing the quality of the IDSR system is necessary as well (8.3.2).

Keywords: monitor, completeness, case reporting, district level, timeliness, identify problems, report, other levels, attributes.
Where should completeness and timeliness of reports be monitored?


100%|█████████▉| 498/499 [33:39<00:03,  3.64s/it]

 Answers: Completeness and timeliness of reports should be monitored at each reporting site and for each level in a communicable disease surveillance system.

Keywords: monitoring completeness reporting sites, measuring timeliness of immediate notification, tracking weekly and monthly reporting, ensuring timely submission of surveillance forms.
How does community-based surveillance contribute to the overall quality of the surveillance system?


100%|██████████| 499/499 [33:44<00:00,  4.06s/it]

 Answers: Community-based surveillance (CBS) contributes to the overall quality of the surveillance system by allowing for event detection and reporting at the community level. This includes monitoring community-based surveillance systems and events from community sources, which is equally important as monitoring at health facilities, districts, and national levels. CBS empowers community members to be involved in the system, increasing ownership and effectiveness. Any community member can serve as a focal person, and they report events or alerts to designated authorities using agreed indicators or through the integrated community health information system (iCHIS).

Keywords: community-based surveillance, CBS, event detection, reporting, community focal persons, public health significance, iCHIS, community empowerment, ownership, community informants.
CPU times: user 1min 56s, sys: 2.28 s, total: 1min 59s
Wall time: 33min 44s


In [17]:
test['Answers'] = results
test["Answers"] = test["Answers"].str.replace("\n", "")
test["Keywords"] = keywords
test['Paragraph_nums'] = paragraph_nums
test['Source files'] = sources

In [18]:
test[test["Answers"].isnull()]

,ID,Question Text,Answers,Keywords,Paragraph_nums,Source files


In [19]:
ID = []
Target = []

for index, row in tqdm(test.iterrows(), total=len(test)):
    ID.append(row["ID"]+"_keywords")
    Target.append(row["Keywords"])
    ID.append(row["ID"]+"_paragraph(s)_number")
    Target.append(row["Paragraph_nums"])
    ID.append(row["ID"]+"_question_answer")
    Target.append(row["Answers"])
    ID.append(row["ID"]+"_reference_document")
    Target.append(row["Source files"][:12])

100%|██████████| 499/499 [00:00<00:00, 10770.90it/s]


In [20]:
sub_df = pd.DataFrame({"ID":ID, "Target": Target})
sub_df.to_csv("submission.csv", index=False)